### Overview

Siren is supposed to let you `project` fields from joined indices into your results table, akin to pulling fields from joined tables in traditional database queries -- e.g. `SELECT t1.*, t2.article.source FROM company as t1 JOIN article as t2 ON t1.id = t2.company`



#### Notebook setup

In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import httpx

client = httpx.Client(
    base_url="https://siren:9220",
    verify=False,
    auth=("sirenadmin", "password"),
    headers={"Content-Type": "application/json"},
)
client

<IPython.core.display.Javascript object>

### Working query

In [3]:
endpoint = "/siren/company/_search"

# Only companies that have been reported on by NYT according to `article` index
body = {
    "size": 1,
    "query": {
        "join": {
            "indices": ["article"],
            "on": ["id", "companies"],
            "request": {
                "query": {"query_string": {"query": 'article.source:"New York Times"'}},
            },
        },
    },
}

client.post(endpoint, json=body).json()

{'pit_id': '32ef5747-ea1e-40c7-97d7-fab1f47a16cb',
 'took': 3,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 921, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'company',
    '_type': 'Company',
    '_id': '6Sy32XcBu9D2HElYQ6bx',
    '_score': 1.0,
    '_ignored': ['deadpooled_date'],
    '_source': {'webpage': '',
     'phone_number': '',
     'location': None,
     'email_address': 'info@favorites.bz',
     'city': None,
     'description': 'Visual Web Directory',
     'homepage_url': 'http://www.favorites.bz',
     'founded_month': 10,
     'revenuecurrency': '',
     'hasstatus': '',
     'one_competitor': 'company/allmyfaves',
     'statecode': None,
     'deadpooled_date': '',
     'url': 'http://www.crunchbase.com/company/favorites',
     'freebaseid': '',
     'number_of_employees': 2,
     'revenue': '',
     'permalink': 'favorites',
     'founded_year': 2012,
     'id': 'company/favorite

<IPython.core.display.Javascript object>

### Bug

In the following query, I expect to see `{'fields': {'article_source': [<value>]}}` in the results, but it is not there.

https://docs.siren.io/siren-federate-user-guide/24/siren-federate/query-dsl.html#siren-federate-query-dsl-project

In [4]:
endpoint = "/siren/company/_search"

# Only companies that have been reported on by NYT
body = {
    "size": 1,
    "query": {
        "join": {
            "indices": ["article"],
            "on": ["id", "companies"],
            "request": {
                "project": [
                    {"field": {"name": "article.source", "alias": "article_source"}}
                ],
                "query": {"query_string": {"query": 'article.source:"New York Times"'}},
            },
        },
    },
}

client.post(endpoint, json=body).json()

{'pit_id': 'd9c0dd9e-dda7-40d8-a387-4e6085bb3098',
 'took': 13,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 921, 'relation': 'eq'},
  'max_score': 1.0,
  'hits': [{'_index': 'company',
    '_type': 'Company',
    '_id': '6Sy32XcBu9D2HElYQ6bx',
    '_score': 1.0,
    '_ignored': ['deadpooled_date'],
    '_source': {'webpage': '',
     'phone_number': '',
     'location': None,
     'email_address': 'info@favorites.bz',
     'city': None,
     'description': 'Visual Web Directory',
     'homepage_url': 'http://www.favorites.bz',
     'founded_month': 10,
     'revenuecurrency': '',
     'hasstatus': '',
     'one_competitor': 'company/allmyfaves',
     'statecode': None,
     'deadpooled_date': '',
     'url': 'http://www.crunchbase.com/company/favorites',
     'freebaseid': '',
     'number_of_employees': 2,
     'revenue': '',
     'permalink': 'favorites',
     'founded_year': 2012,
     'id': 'company/favorit

<IPython.core.display.Javascript object>